<p style="font-family: Arial; font-size:3.0em;color:yellow; font-style:bold"><br>
Outer approximation and a location problem
</p><br>

<p style="text-align:center; font-family: Arial; font-size:2.00em;color:cyan; font-style:bold"><br>
Dr. Hoa Bui
</p><br>

<p style="text-align:center; font-family: Arial; font-size:1.00em;color:white; font-style:bold">Originally created for: </p>

<p style="text-align:center; font-family: Arial; font-size:1.00em;color:green; font-style:bold">
Mathematical Optimisation: Theory and Application, AMSI Summer School 2026</p>

<br>
<br>

# Solve the following quadratic problem
Given $n$ location in $\mathbb{R}^2$, select $k$ locations such that the sum distance is maximised.

Mathematical formulation

\begin{align*}
\max\quad &\sum_{i=1}^n q_{ij} \times x_i\times x_j\\
\text{s.t.} \quad & \sum_{i=1}^n x_i=k_t,\\
& x_i\in\{0,1\}
\end{align*}
Here $x_i = 1$ if location $i$ is selected, and $x_i=0$ otherwise.

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import Model, GRB
import math
from typing import List, Tuple
from Linearisationtechnique import qp_model, second_linear
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
import plotly
from numba import njit
import plotly.graph_objects as go
from termcolor import colored

1. numba: https://numba.pydata.org
2. sklearn.cluster: https://scikit-learn.org/stable/modules/clustering.html

In [2]:
import warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=NumbaPendingDeprecationWarning)
# plotly.graph_objs._scattergl.Scattergl

## Create functions to generated distance matrix from locations data

In [3]:
# calculate distance
@njit
def distance(u: np.array, v: np.array) -> float:
    """
    Calculates the Euclidean distance between two points in a 2D space.

    Args:
        u: A numpy array representing the coordinates of the first point.
        v: A numpy array representing the coordinates of the second point.

    Returns:
        The Euclidean distance between the two points.
    """
    # The formula for Euclidean distance is sqrt(sum((u_i - v_i)^2))
    return np.sqrt(np.sum((u - v) ** 2))


@njit
def distance_matrix(locations: List[np.array]) -> np.array:
    """
    Computes a symmetric distance matrix for a list of locations.

    Args:
        locations: A list or array of coordinates, where each element
                  represents a point in 2D space.

    Returns:
        A symmetric n x n numpy array where the element (i, j) is the
        Euclidean distance between location i and location j.
    """
    # Get the number of locations
    n = len(locations)

    # Initialize an n x n matrix with zeros
    matrix = np.zeros((n, n))

    # Iterate through each pair of locations to compute the distance
    for i in range(n):
        # Start from i + 1 to calculate only the upper triangle
        for j in range(i + 1, n):
            # Calculate the distance between point i and point j
            dist = distance(locations[i], locations[j])
            # Assign the distance to both (i, j) and (j, i) for symmetry
            matrix[i, j] = dist
            matrix[j, i] = dist

    return matrix

# Build the outer approximation model
$$
\begin{aligned}
\max \quad & \theta \\
\text{s.t.} \quad & \sum_{i=1}^n x_i = k \\
\quad & \theta \le \langle Qy, x-y \rangle + \tfrac{1}{2}\langle Qy, y \rangle, \quad \forall y \in C\quad (*) \\
& x_i \in \{0, 1\} \\
& \theta \le UB.
\end{aligned}
$$

## Input data and setup model

In [4]:
# Set random seed for reproducibility
np.random.seed(42)

# number of nodes
n = 100
k = math.floor(n / 2)

# generate n locations
location = [np.random.randint(1, 50, size=2) for i in range(n)]

# get distance matrix
matrix = distance_matrix(location)

# get upperbound for theta
upperbound = 1 / 2 * sum(matrix[i, j] for i in range(n) for j in range(n))


# get starting point
xk = np.zeros(n)
xk[:k] = 1

## Master Problem


Mater problem
\begin{align*}
\max\quad &\theta\\
\text{s.t.} \quad 
& \sum_{i=1}^n x_i=k\\
& x_i\in\{0,1\}\\
&\theta \le UB
\end{align*}

In [5]:
# setup model
m = Model("OuterApprox")

# set variables
x = m.addVars(n, vtype=GRB.BINARY, name="x")

# set theta
theta = m.addVar(name="theta", ub=upperbound, vtype=GRB.CONTINUOUS)

# add constraints: cardinality constraint
m.addConstr(gp.quicksum(x[i] for i in range(n)) == k)

# objective function
m.setObjective(theta, GRB.MAXIMIZE)

Set parameter Username
Set parameter LicenseID to value 2630841
Academic license - for non-commercial use only - expires 2026-03-04


## Calculate objective function, and gradient

$$
f(x) = \tfrac{1}{2} \langle Qx,x\rangle
$$
and
$$
\nabla f(x) = Qx.
$$

In [6]:
def f(x: np.ndarray, matrix: np.ndarray) -> float:
    """
    Calculates the value of the quadratic objective function.

    The function computes 0.5 * x' * Q * x, where Q is the distance matrix.

    Args:
        x: A numpy array representing the solution vector.
        matrix: The symmetric distance matrix (Q).

    Returns:
        The calculated value of the objective function as a float.
    """
    return x.dot(matrix).dot(x) / 2


def df(x: np.ndarray, matrix: np.ndarray) -> np.ndarray:
    """
    Calculates the gradient of the quadratic objective function.

    The gradient is computed as Q * x.

    Args:
        x: A numpy array representing the solution vector.
        matrix: The symmetric distance matrix (Q).

    Returns:
        A numpy array representing the gradient of the function.
    """
    return x.dot(matrix)

## Define cutting planes

We add constraints (*) one by one where $y$ is the solution of the previous iteration
$$
\theta \le \langle Qy,x-y\rangle + \tfrac{1}{2}\langle Qy,y\rangle = \langle Qy,x\rangle - \tfrac{1}{2}\langle Qy,y\rangle
$$
$\textbf{Stopping condition}$: upperbound $=$ lowerbound 

lowerbound $= f(x^k)$

upperbound $= \theta^k$

In [7]:
# get lower bound
lowerbound = f(xk, matrix)

# set up first step
num_iter = 0
m.setParam("OutputFlag", 0)

# start the OA
while upperbound - lowerbound > 0.01 and num_iter < 100:

    # get gradient
    dfx = df(xk, matrix)

    # update the cutting plane
    m.addConstr(theta <= gp.quicksum(dfx[i] * x[i] for i in range(n)) - f(xk, matrix))

    # solve the new model
    m.optimize()

    # get new vector
    xk = np.array([x[i].X for i in range(n)])

    # update lower bound
    lowerbound = f(xk, matrix)

    # update upper bound
    upperbound = m.ObjVal

    # update steps
    num_iter += 1

print(colored("Number of iteration:", "green", attrs=["bold"]), num_iter)
print(colored("Objective value:", "green", attrs=["bold"]), m.ObjVal)
print(colored("Solution:", "green", attrs=["bold"]), [x[i].X for i in range(n)])

Number of iteration: 8
Objective value: 38048.76234955194
Solution: [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0]


In [8]:
# Outer Approximation Function
def outer_approximation(
    n: int,
    matrix: np.ndarray,
    k: int,
    timeout: float = 3600,
    verbose: bool = True,
) -> Tuple[float, float, List[float], float]:
    """
    Solves the quadratic location problem using the Outer Approximation algorithm.

    This function iteratively builds a master problem by adding cutting planes
    derived from the gradient of the objective function at the solution of
    the previous iteration.

    Args:
        n: The total number of locations.
        matrix: The symmetric distance matrix (Q).
        k: The number of locations to select.
        timeout: The maximum time in seconds for the solver.
        verbose: If True, prints progress at each iteration.

    Returns:
        A tuple containing:
            - The final objective value (upper bound).
            - The total solver runtime.
            - The solution vector (list of binary values).
            - The final optimality gap as a percentage.
    """
    m = None
    try:
        # get upperbound for theta
        upperbound = 1 / 2 * sum(matrix[i, j] for i in range(n) for j in range(n))

        # get starting point
        xk = np.zeros(n)
        xk[:k] = 1

        # setup model
        m = Model("OuterApprox")
        m.setParam("TimeLimit", timeout)
        m.setParam("OutputFlag", 0)

        # set variables
        x = m.addVars(n, vtype=GRB.BINARY, name="x")

        # set theta
        theta = m.addVar(name="theta", ub=upperbound, vtype=GRB.CONTINUOUS)

        # add constraints
        m.addConstr(gp.quicksum(x[i] for i in range(n)) == k)

        # objective function
        m.setObjective(theta, GRB.MAXIMIZE)

        # get lower bound
        lowerbound = f(xk, matrix)

        # set up first step
        num_iter = 0

        # get time solve
        time_solve = 0

        while (upperbound - lowerbound > 0.1 or num_iter < 1) and num_iter < 100:
            # get gradient
            dfx = df(xk, matrix)

            # update the cutting plane
            m.addConstr(
                theta <= gp.quicksum(dfx[i] * x[i] for i in range(n)) - lowerbound
            )

            # solve the new model
            m.optimize()

            # get new vector
            xk = np.array([x[i].X for i in range(n)])

            # update lower bound
            lowerbound = f(xk, matrix)

            # update upper bound
            upperbound = m.ObjVal

            # update steps
            num_iter += 1

            # update timesolve
            time_solve += m.Runtime

            # print progress
            if verbose:
                print(
                    f"Iteration {colored(num_iter, 'yellow')}: "
                    f"Upper Bound = {colored(f'{upperbound:.2f}', 'cyan')}, "
                    f"Lower Bound = {colored(f'{lowerbound:.2f}', 'magenta')}"
                )

        return (
            m.ObjVal,
            time_solve,
            [v.X for v in x.values()],
            (upperbound - lowerbound) / upperbound * 100 if upperbound > 0 else 0,
        )
    finally:
        if m:
            m.dispose()

# Testing against the second linearisation formulation, and QP solver

In [9]:
# Define the problem sizes (number of nodes) to test
N = [20, 40, 50, 60, 80, 90, 100]

# Set a time limit for each solver run to ensure a fair comparison
time_constraint = 10  # in second

# Initialize a dictionary to store the results from the comparison experiment
comparision = {
    "n": [],
    "Solutions": [],
    "Runtime": [],
    "Optimal Value": [],
    "Optimality Gap": [],
}
# Loop through each defined problem size
for n in N:
    # Set a random seed to ensure the generated data is the same for each run
    np.random.seed(10)

    # Determine the number of locations to select (half of the total)
    k = math.floor(n / 2)

    # Generate 'n' random 2D coordinates for the locations
    locations = [np.random.randint(1, 50, size=2) for i in range(n)]

    # Compute the distance matrix for the generated locations
    matrix = distance_matrix(locations=locations)

    # --- Run each of the three solvers and collect their performance metrics ---
    # 1. Solve using Gurobi's native Quadratic Programming (QP) solver
    qp_value, qp_runtime, _, qp_gap = qp_model(
        n=n,
        matrix=matrix,
        k=k,
        timeout=time_constraint,
    )
    # 2. Solve using the Outer Approximation algorithm
    oa_value, oa_runtime, _, oa_gap = outer_approximation(
        n=n,
        matrix=matrix,
        k=k,
        timeout=time_constraint,
        verbose=False,
    )
    # 3. Solve using the second linearization technique
    sl_value, second_linear_runtime, _, sl_gap = second_linear(
        n=n,
        matrix=matrix,
        k=k,
        timeout=time_constraint,
    )

    # --- Append the results from this iteration to the comparison dictionary ---

    # Add the problem size (n) for each of the three solvers
    comparision["n"].extend(np.repeat(n, 3))
    # Add the names of the solvers
    comparision["Solutions"].extend(
        ["(QP) Solver", "Second Linear", "Outer Approximation"]
    )
    # Add the runtime for each solver
    comparision["Runtime"].extend([qp_runtime, second_linear_runtime, oa_runtime])
    # Add the final objective value for each solver
    comparision["Optimal Value"].extend([qp_value, sl_value, oa_value])
    # Add the final optimality gap for each solver
    comparision["Optimality Gap"].extend([qp_gap, sl_gap, oa_gap])

# Convert the results dictionary into a pandas DataFrame
df_comparision = pd.DataFrame(comparision)

Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10


In [10]:
# print the table
df_comparision

,n,Solutions,Runtime,Optimal Value,Optimality Gap
0,20,(QP) Solver,0.056031,1430.534329,0.000000e+00
1,20,Second Linear,0.058322,1430.534329,0.000000e+00
2,20,Outer Approximation,0.002676,1430.534329,0.000000e+00
3,40,(QP) Solver,0.700127,5623.106550,0.000000e+00
4,40,Second Linear,0.600173,5623.106550,0.000000e+00
5,40,Outer Approximation,0.006699,5623.106550,1.617424e-14
6,50,(QP) Solver,10.001849,9114.780780,2.929327e-01
7,50,Second Linear,4.193669,9114.780780,0.000000e+00
8,50,Outer Approximation,0.003409,9114.780780,0.000000e+00
9,60,(QP) Solver,10.002786,13323.426731,3.580556e-01


In [11]:
# visualised the results
fig = px.line(
    data_frame=df_comparision,
    y="Optimality Gap",
    x="n",
    color="Solutions",
    template="none",
    height=600,
    markers=True,
    title=f"Comparision of each algorithm's optimal solutions, provided the timeout = {time_constraint} (s).",
)
fig.show()

In [12]:
# visualised the results using plotly
fig = px.line(
    data_frame=df_comparision,
    y="Runtime",
    x="n",
    color="Solutions",
    template="none",
    height=600,
    markers=True,
    title=f"Comparision of each algorithm's run time, provided the timeout = {time_constraint} (s).",
)
fig.show()

# Real world coordinates

Demonstrated data is available from https://www.kaggle.com/competitions/santas-stolen-sleigh/data.

Provided the coordinations in the given dataset, we will extract data from the USA. Then, we will:

1. Find k data points such that the total distance from those data points is maximised, using Outer Approximation.

2. We apply K-mean clustering (https://neptune.ai/blog/k-means-clustering) to divide the data into multiple regions. We will use the Outer Approximation algorithm above to pick up k data points from each cluster to maximise the sum distance.

## Without Clusters
### Get data

In [13]:
# get the data
locations = pd.read_csv("data/locations/gifts.csv")

# print our data
locations

,GiftId,Latitude,Longitude,Weight
0,1,16.345769,6.303545,1.000000
1,2,12.494749,28.626396,15.524480
2,3,27.794615,60.032495,8.058499
3,4,44.426992,110.114216,1.000000
4,5,-69.854088,87.946878,25.088892
...,...,...,...,...
99995,99996,-86.087115,-19.991474,10.927676
99996,99997,40.259124,-110.689749,9.347134
99997,99998,42.393016,0.011825,1.000000
99998,99999,-75.919193,-10.193532,1.000000


In [14]:
# Create a new collumn to combine latitude and longitude
locations["lat_long"] = locations.apply(
    lambda row: np.array([row["Latitude"], row["Longitude"]]), axis=1
).values.tolist()

# Get the coordinate in a list as input data
locations_coord = locations["lat_long"].values.tolist()

### Solve with only Outer Approximation

We solve the problem of selecting $50$ locations out of the first $1000$ locations

In [15]:
# Select a subset of sel_n locations for the experiment
sel_n = 1000
locations_reduced = locations_coord[:sel_n]

# Get distance matrix from the locations
matrix = distance_matrix(locations=locations_reduced)

# We select k locations out of sel_n locations
k = 50

# Use Outer Approximation to solve the optimisation problem
oa_value, oa_runtime, oa_sol, oa_gap = outer_approximation(
    n=sel_n, matrix=matrix, k=k, timeout=3600
)

Set parameter TimeLimit to value 3600
Iteration 1: Upper Bound = 338191.92, Lower Bound = 239232.00
Iteration 2: Upper Bound = 306261.81, Lower Bound = 247719.16
Iteration 3: Upper Bound = 278449.01, Lower Bound = 235612.34
Iteration 4: Upper Bound = 276490.34, Lower Bound = 260504.98
Iteration 5: Upper Bound = 265825.54, Lower Bound = 237180.93
Iteration 6: Upper Bound = 264757.96, Lower Bound = 256323.90
Iteration 7: Upper Bound = 263121.34, Lower Bound = 260192.68
Iteration 8: Upper Bound = 262577.07, Lower Bound = 260125.51
Iteration 9: Upper Bound = 262231.41, Lower Bound = 261217.96
Iteration 10: Upper Bound = 261824.86, Lower Bound = 260759.46
Iteration 11: Upper Bound = 261544.08, Lower Bound = 261035.95
Iteration 12: Upper Bound = 261463.71, Lower Bound = 261179.11
Iteration 13: Upper Bound = 261439.78, Lower Bound = 261289.70
Iteration 14: Upper Bound = 261401.05, Lower Bound = 261261.49
Iteration 15: Upper Bound = 261329.21, Lower Bound = 261259.49
Iteration 16: Upper Bound 

### Visualise the results

In [16]:
# Create a copy of the DataFrame for visualization
locations_sample = locations.head(sel_n).copy()

# Add the solution vector to the new DataFrame
locations_sample["solution"] = oa_sol
# Map the numeric solution to a descriptive status for a clearer legend
locations_sample["Status"] = locations_sample["solution"].map(
    {1.0: "Selected", 0.0: "Not Selected"}
)

# Create the geographic scatter plot
fig = px.scatter_geo(
    locations_sample,
    lat="Latitude",
    lon="Longitude",
    color="Status",  # Use the new 'Status' column for coloring
    # Map the descriptive status to specific colors
    color_discrete_map={
        "Selected": "red",
        "Not Selected": "blue",
    },
    title="Optimal Locations for Max-sum Problem",
)
fig.show()

## With Cluster

Extract North America data points

Country-bounding-boxes: https://gist.github.com/graydon/11198540

Extracted from http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip


In [17]:
# Extract North America data points
locations_us = locations.query(
    "Longitude > -171.791110603 & Longitude < -66.96466 & Latitude > 18.91619 & Latitude < 71.3577635769"
).reset_index(drop=True)
locations_us

,GiftId,Latitude,Longitude,Weight,lat_long
0,6,53.567970,-71.359308,38.000151,"[53.5679698071, -71.3593080866]"
1,18,48.750631,-124.572001,6.884479,"[48.7506310206, -124.572001202]"
2,19,34.222105,-99.390863,45.808683,"[34.2221051613, -99.3908630501]"
3,32,58.916676,-128.640284,8.922934,"[58.9166758589, -128.640284481]"
4,37,50.242051,-95.537747,1.000000,"[50.242050756, -95.5377472093]"
...,...,...,...,...,...
13090,99984,39.327327,-79.523378,1.000000,"[39.3273265377, -79.5233776312]"
13091,99986,53.607246,-127.001301,1.000000,"[53.6072460325, -127.001300958]"
13092,99988,28.321648,-113.083080,27.702490,"[28.321648198, -113.08308041]"
13093,99990,57.684642,-94.381203,11.262914,"[57.6846424542, -94.3812026137]"


### Get clustering from Kmean

In [18]:
# Get the coordinates for clustering
locations_coord_us = locations_us["lat_long"].values.tolist()

# Create some clusters using Kmeans clustering
clustering = KMeans(n_clusters=50, random_state=42, n_init="auto").fit(
    locations_coord_us
)
np.unique(clustering.labels_)
locations_us["cluster_id"] = clustering.labels_

# Drop outlier with value -1 and 17
# locations_us = locations_us[~locations_us.cluster_id.isin([-1, 17])].copy()
locations_us["cluster_id"] = locations_us["cluster_id"].astype(str)

# Count how many data points per clusters
locations_us_count = (
    locations_us.groupby("cluster_id")["cluster_id"]
    .agg(nobs="count")
    .reset_index(drop=False)
)
locations_us = locations_us.merge(locations_us_count, how="left", on="cluster_id")

# Demonstrate through a small number of clusters
# locations_us = locations_us.query('nobs%4==0')
print(locations_us.cluster_id.unique())
locations_us

['4' '5' '25' '33' '22' '23' '26' '40' '12' '48' '15' '11' '20' '17' '43'
 '35' '37' '2' '3' '42' '34' '13' '9' '41' '49' '8' '29' '14' '39' '24'
 '46' '6' '21' '27' '19' '45' '10' '16' '38' '0' '28' '18' '1' '36' '31'
 '7' '32' '44' '30' '47']


,GiftId,Latitude,Longitude,Weight,lat_long,cluster_id,nobs
0,6,53.567970,-71.359308,38.000151,"[53.5679698071, -71.3593080866]",4,280
1,18,48.750631,-124.572001,6.884479,"[48.7506310206, -124.572001202]",5,228
2,19,34.222105,-99.390863,45.808683,"[34.2221051613, -99.3908630501]",25,331
3,32,58.916676,-128.640284,8.922934,"[58.9166758589, -128.640284481]",33,297
4,37,50.242051,-95.537747,1.000000,"[50.242050756, -95.5377472093]",22,338
...,...,...,...,...,...,...,...
13090,99984,39.327327,-79.523378,1.000000,"[39.3273265377, -79.5233776312]",20,379
13091,99986,53.607246,-127.001301,1.000000,"[53.6072460325, -127.001300958]",33,297
13092,99988,28.321648,-113.083080,27.702490,"[28.321648198, -113.08308041]",48,262
13093,99990,57.684642,-94.381203,11.262914,"[57.6846424542, -94.3812026137]",22,338


### Visualise the clustering

In [19]:
# plotly.graph_objs._scattergl.Scattergl
fig = px.scatter_geo(
    locations_us,
    lat=locations_us.Latitude,
    lon=locations_us.Longitude,
    color=locations_us.cluster_id,
)
fig.update_layout(geo_scope="north america", geo=dict(showland=True))

fig.show()

## We write a function for outer approximation to solve the optimisation problem of selecting 1 location among each cluster

In [20]:
# Outer Approximation Function
def outer_approximation_location(
    location_df: pd.DataFrame,
    k: int,
    timeout: float = 3600,
    verbose: bool = True,
) -> Tuple[pd.DataFrame, dict]:
    """
    Solves the clustered location problem using the Outer Approximation algorithm.

    This function selects 'k' locations from each cluster to maximize the total
    sum of distances, using outer approximation.

    Args:
        location_df: DataFrame containing location data, including 'lat_long'
                     and 'cluster_id' columns.
        k: The number of locations to select from each cluster.
        timeout: The maximum time in seconds for the solver.
        verbose: If True, prints progress at each iteration.

    Returns:
        A tuple containing:
            - The updated DataFrame with a new 'opt_sol' column indicating
              the selected locations.
            - A dictionary with the cumulative runtime at each iteration.
    """
    m = None
    try:
        n = location_df.shape[0]
        locations_coord = location_df["lat_long"].values.tolist()
        cluster_group = location_df["cluster_id"].values.tolist()

        # get distance matrix
        matrix = distance_matrix(locations_coord)

        # get upperbound for theta
        upperbound = 1 / 2 * sum(matrix[i, j] for i in range(n) for j in range(n))

        # get starting point
        xk = np.zeros(n)
        xk[: (k * len(np.unique(cluster_group)))] = 1

        # setup model
        m = Model("OA_Location")
        m.setParam("TimeLimit", timeout)
        m.setParam("OutputFlag", 0)

        # set variables
        x = m.addVars(n, vtype=GRB.BINARY, name="x")

        # set theta
        theta = m.addVar(name="theta", ub=upperbound, vtype=GRB.CONTINUOUS)

        # add constraints
        m.addConstrs(
            (
                gp.quicksum(x[i] for i in range(n) if cluster_group[i] == cl) == k
                for cl in np.unique(cluster_group)
            ),
            name="k_per_cluster",
        )

        # objective function
        m.setObjective(theta, GRB.MAXIMIZE)

        # get lower bound
        lowerbound = f(xk, matrix)

        # set up first step
        num_iter = 0
        time_solve = 0
        time_solve_cummulative = {
            "Iteration": [],
            "Total time (s)": [],
        }
        while ((upperbound - lowerbound) > 0.0001 or (num_iter < 1)) and (
            num_iter < 100
        ):
            # get gradient
            dfx = df(xk, matrix)

            # update the cutting plane
            m.addConstr(
                theta <= gp.quicksum(dfx[i] * x[i] for i in range(n)) - f(xk, matrix)
            )

            # solve the new model
            m.optimize()

            # get new vector
            xk = np.array([x[i].X for i in range(n)])

            # update lower bound
            lowerbound = f(xk, matrix)

            # update upper bound
            upperbound = m.ObjVal

            # update steps
            num_iter += 1
            time_solve += m.Runtime
            time_solve_cummulative["Iteration"].append(num_iter)
            time_solve_cummulative["Total time (s)"].append(time_solve)

            if verbose:
                print(
                    f"Iteration {colored(num_iter, 'yellow')}: "
                    f"Upper Bound = {colored(f'{upperbound:.2f}', 'cyan')}, "
                    f"Lower Bound = {colored(f'{lowerbound:.2f}', 'magenta')}"
                )

        location_df["opt_sol"] = [x[i].X for i in range(n)]
        location_df["opt_sol"] = location_df["opt_sol"].astype(str)
        print(f"Optimal value: {m.ObjVal}, within {num_iter}  iteration(s).")
        return location_df, time_solve_cummulative
    finally:
        if m:
            m.dispose()

### We use Outer Approximation to solve the problem

In [21]:
oa_location, time_solve_cummulative = outer_approximation_location(
    locations_us,
    k=2,
    timeout=100,
    verbose=False,
)
oa_location

Set parameter TimeLimit to value 100
Optimal value: 204477.03358215684, within 6  iteration(s).


,GiftId,Latitude,Longitude,Weight,lat_long,cluster_id,nobs,opt_sol
0,6,53.567970,-71.359308,38.000151,"[53.5679698071, -71.3593080866]",4,280,0.0
1,18,48.750631,-124.572001,6.884479,"[48.7506310206, -124.572001202]",5,228,0.0
2,19,34.222105,-99.390863,45.808683,"[34.2221051613, -99.3908630501]",25,331,0.0
3,32,58.916676,-128.640284,8.922934,"[58.9166758589, -128.640284481]",33,297,0.0
4,37,50.242051,-95.537747,1.000000,"[50.242050756, -95.5377472093]",22,338,0.0
...,...,...,...,...,...,...,...,...
13090,99984,39.327327,-79.523378,1.000000,"[39.3273265377, -79.5233776312]",20,379,0.0
13091,99986,53.607246,-127.001301,1.000000,"[53.6072460325, -127.001300958]",33,297,0.0
13092,99988,28.321648,-113.083080,27.702490,"[28.321648198, -113.08308041]",48,262,0.0
13093,99990,57.684642,-94.381203,11.262914,"[57.6846424542, -94.3812026137]",22,338,0.0


### Visualise the results

In [22]:
fig = px.scatter(
    oa_location.query('opt_sol!="1.0"'),
    x="Longitude",
    y="Latitude",
    color="cluster_id",
    symbol="cluster_id",
    template="none",
)

max_opt_distance_df = oa_location.query('opt_sol=="1.0"')


fig.add_trace(
    go.Scatter(
        x=max_opt_distance_df.Longitude,
        y=max_opt_distance_df.Latitude,
        name="Optimal data points \n- Outer Approximation",
        mode="markers",
        text=max_opt_distance_df.cluster_id,
        marker=dict(color="black", size=10),
    )
)

fig.show()

### Visualise the cummulative runtime

In [23]:
fig = px.scatter(
    pd.DataFrame(time_solve_cummulative),
    x="Iteration",
    y="Total time (s)",
    title="Cummulative runtime",
    template="none",
)
fig.update_traces(mode="lines+markers")
fig.show()